In [12]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
load_dotenv()
import folium

In [13]:
#get mrt data
mrt_df = pd.read_csv('../../data/Isochrone_data/mrt_station_colours.csv', index_col=0)
mrt_df.head()
coords = mrt_df[['Latitude', 'Longitude']].values.tolist()

# run this to start OTP: java -Xmx6G -jar otp-2.5.0-shaded.jar --load /Users/ethan/Documents/GitHub/Public-Transportation-In-Singapore/Backend/otp/  --serve

# Sample API request to the OTP isochrone endpoint
# response = requests.get("http://localhost:8080/otp/traveltime/isochrone?batch=true&location=1.3521,103.895&time=2023-04-12T10:19:03%2B02:00&modes=WALK,TRANSIT&arriveBy=false&cutoff=30M17S")
# isochrone_geojson = response.json()


Write a function for fetching isochrome. Configure params here

In [14]:
def fetch_isochrone(lat, lon, modes, cutoff="20M01S"):
    """Fetch isochrone GeoJSON for a given location."""
    url = "http://localhost:8080/otp/traveltime/isochrone"
    params = {
        "batch": "false",
        "location": f"{lat},{lon}",
        "time": "2024-04-02T10:00:00+02:00",
        "modes": modes,
        "cutoff": cutoff,
        "maxWalkDistance": 2,
        "walkReluctance": 4
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch isochrone for {lat},{lon}. Status code: {response.status_code}")
        return None

x = fetch_isochrone(1.293657725, 103.8550812, modes = "TRANSIT")
features = x['features'][0]['geometry']['coordinates'][0][0]
print(features)

[[103.845496, 1.295226], [103.84668, 1.296167], [103.846697, 1.294628], [103.846701, 1.294611], [103.847241, 1.293373], [103.847261, 1.292813], [103.847243, 1.291577], [103.84745, 1.291014], [103.847505, 1.29004], [103.847781, 1.289215], [103.846788, 1.287524], [103.848479, 1.288315], [103.848817, 1.287754], [103.848986, 1.287417], [103.848482, 1.28562], [103.850278, 1.28628], [103.850711, 1.286051], [103.851461, 1.285618], [103.851631, 1.285172], [103.852077, 1.284816], [103.852584, 1.284326], [103.853584, 1.283819], [103.8522, 1.282143], [103.853877, 1.283708], [103.855168, 1.283819], [103.855556, 1.285498], [103.855568, 1.285618], [103.855676, 1.285978], [103.85697, 1.287417], [103.857475, 1.287595], [103.857691, 1.287633], [103.858337, 1.287417], [103.859129, 1.287272], [103.859274, 1.287349], [103.860404, 1.286748], [103.861073, 1.286388], [103.862152, 1.287417], [103.862872, 1.287936], [103.864312, 1.288857], [103.864671, 1.287794], [103.865241, 1.289215], [103.865238, 1.289782],

DATA NEEDED:

Isochrones: 10min-60min, 1min interval (ideal) else 5min interval

1. BICYCLE
2. BUS+ WALK
3. RAIL+ WALK
4. TRANSIT + WALK
5. CAR

Set up function for saving all isochrones by transport mode

In [15]:
def fetch_all_isochrones(df, mode, interval):
    """Fetch and append bicycle isochrones for each MRT station and specified time intervals."""
    # Define the cutoff intervals (in seconds) - lets us choose 1, 5, or 10 minute intervals later on
    cutoff_intervals = [f"{x}M" for x in range(10, 61, interval)]
    
    # Iterate over the cutoff intervals
    for interval in cutoff_intervals:
        # Define a column name for the interval
        column_name = f"isochrone_{interval}"
        # Initialize the column with empty lists
        df[column_name] = None
        
        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            # Fetch the isochrone GeoJSON for the current MRT station and cutoff interval
            isochrone_geojson = fetch_isochrone(row['Latitude'], row['Longitude'], modes=mode, cutoff=interval )
            
            # Check if the response is not None
            if isochrone_geojson:
                # Extract the coordinates of the first feature (assuming single isochrone feature per request)
                if 'features' in isochrone_geojson and len(isochrone_geojson['features']) > 0:
                    features = isochrone_geojson['features'][0]['geometry']['coordinates'][0][0]
                    # Assign the extracted features to the corresponding DataFrame cell
                    df.at[index, column_name] = features
                print(f"Fetched isochrone for interval {interval}, MRT: {row['MRT.Name']}")
            else:
                print(f"Failed to fetch isochrone for interval {interval}, MRT: {row['MRT.Name']}")
    
    return df

# Note: Ensure that mrt_df already exists and contains 'MRT.Name', 'Latitude', and 'Longitude' columns


In [18]:
# Apply the function to the mrt_df DataFrame
bicycle_isochrones = mrt_df.copy()
bicycle_isochrones = fetch_all_isochrones(bicycle_isochrones, "BICYCLE", interval=5)
bicycle_isochrones.to_json('../../data/Isochrone_data/bicycle_isochrones_new.json', orient='records', lines=True, force_ascii=False)

Fetched isochrone for interval 10M, MRT: ESPLANADE MRT STATION
Fetched isochrone for interval 10M, MRT: PAYA LEBAR MRT STATION
Fetched isochrone for interval 10M, MRT: DHOBY GHAUT MRT STATION
Fetched isochrone for interval 10M, MRT: DAKOTA MRT STATION
Fetched isochrone for interval 10M, MRT: LAVENDER MRT STATION
Fetched isochrone for interval 10M, MRT: RENJONG LRT STATION
Fetched isochrone for interval 10M, MRT: DOVER MRT STATION
Fetched isochrone for interval 10M, MRT: HOUGANG MRT STATION
Fetched isochrone for interval 10M, MRT: PHOENIX LRT STATION
Fetched isochrone for interval 10M, MRT: ALJUNIED MRT STATION
Fetched isochrone for interval 10M, MRT: COVE LRT STATION
Fetched isochrone for interval 10M, MRT: PASIR RIS MRT STATION
Fetched isochrone for interval 10M, MRT: ADMIRALTY MRT STATION
Fetched isochrone for interval 10M, MRT: KEMBANGAN MRT STATION
Fetched isochrone for interval 10M, MRT: PUNGGOL POINT LRT STATION
Fetched isochrone for interval 10M, MRT: MARSILING MRT STATION
Fetch

In [19]:
bus_isochrones = mrt_df.copy()
bus_isochrones = fetch_all_isochrones(bus_isochrones,mode="BUS", interval=5)
bus_isochrones.to_json('../../data/Isochrone_data/bus_isochrones_new.json', orient='records', lines=True, force_ascii=False)

Fetched isochrone for interval 10M, MRT: ESPLANADE MRT STATION
Fetched isochrone for interval 10M, MRT: PAYA LEBAR MRT STATION
Fetched isochrone for interval 10M, MRT: DHOBY GHAUT MRT STATION
Fetched isochrone for interval 10M, MRT: DAKOTA MRT STATION
Fetched isochrone for interval 10M, MRT: LAVENDER MRT STATION
Fetched isochrone for interval 10M, MRT: RENJONG LRT STATION
Fetched isochrone for interval 10M, MRT: DOVER MRT STATION
Fetched isochrone for interval 10M, MRT: HOUGANG MRT STATION
Fetched isochrone for interval 10M, MRT: PHOENIX LRT STATION
Fetched isochrone for interval 10M, MRT: ALJUNIED MRT STATION
Fetched isochrone for interval 10M, MRT: COVE LRT STATION
Fetched isochrone for interval 10M, MRT: PASIR RIS MRT STATION
Fetched isochrone for interval 10M, MRT: ADMIRALTY MRT STATION
Fetched isochrone for interval 10M, MRT: KEMBANGAN MRT STATION
Fetched isochrone for interval 10M, MRT: PUNGGOL POINT LRT STATION
Fetched isochrone for interval 10M, MRT: MARSILING MRT STATION
Fetch

In [20]:
mrt_isochrones = mrt_df.copy()
mrt_isochrones = fetch_all_isochrones(mrt_isochrones,mode="SUBWAY", interval=5)
mrt_isochrones.to_json('../../data/Isochrone_data/mrt_isochrones_new.json', orient='records', lines=True, force_ascii=False)

Fetched isochrone for interval 10M, MRT: ESPLANADE MRT STATION
Fetched isochrone for interval 10M, MRT: PAYA LEBAR MRT STATION
Fetched isochrone for interval 10M, MRT: DHOBY GHAUT MRT STATION
Fetched isochrone for interval 10M, MRT: DAKOTA MRT STATION
Fetched isochrone for interval 10M, MRT: LAVENDER MRT STATION
Fetched isochrone for interval 10M, MRT: RENJONG LRT STATION
Fetched isochrone for interval 10M, MRT: DOVER MRT STATION
Fetched isochrone for interval 10M, MRT: HOUGANG MRT STATION
Fetched isochrone for interval 10M, MRT: PHOENIX LRT STATION
Fetched isochrone for interval 10M, MRT: ALJUNIED MRT STATION
Fetched isochrone for interval 10M, MRT: COVE LRT STATION
Fetched isochrone for interval 10M, MRT: PASIR RIS MRT STATION
Fetched isochrone for interval 10M, MRT: ADMIRALTY MRT STATION
Fetched isochrone for interval 10M, MRT: KEMBANGAN MRT STATION
Fetched isochrone for interval 10M, MRT: PUNGGOL POINT LRT STATION
Fetched isochrone for interval 10M, MRT: MARSILING MRT STATION
Fetch

In [21]:
public_isochrones = mrt_df.copy()
public_isochrones = fetch_all_isochrones(public_isochrones,mode="TRANSIT", interval=5)
public_isochrones.to_json('../../data/Isochrone_data/public_isochrones_new.json', orient='records', lines=True, force_ascii=False)

Fetched isochrone for interval 10M, MRT: ESPLANADE MRT STATION
Fetched isochrone for interval 10M, MRT: PAYA LEBAR MRT STATION
Fetched isochrone for interval 10M, MRT: DHOBY GHAUT MRT STATION
Fetched isochrone for interval 10M, MRT: DAKOTA MRT STATION
Fetched isochrone for interval 10M, MRT: LAVENDER MRT STATION
Fetched isochrone for interval 10M, MRT: RENJONG LRT STATION
Fetched isochrone for interval 10M, MRT: DOVER MRT STATION
Fetched isochrone for interval 10M, MRT: HOUGANG MRT STATION
Fetched isochrone for interval 10M, MRT: PHOENIX LRT STATION
Fetched isochrone for interval 10M, MRT: ALJUNIED MRT STATION
Fetched isochrone for interval 10M, MRT: COVE LRT STATION
Fetched isochrone for interval 10M, MRT: PASIR RIS MRT STATION
Fetched isochrone for interval 10M, MRT: ADMIRALTY MRT STATION
Fetched isochrone for interval 10M, MRT: KEMBANGAN MRT STATION
Fetched isochrone for interval 10M, MRT: PUNGGOL POINT LRT STATION
Fetched isochrone for interval 10M, MRT: MARSILING MRT STATION
Fetch

In [9]:
car_isochrones = mrt_df.copy()
car_isochrones = fetch_all_isochrones(car_isochrones,mode="CAR", interval=5)
car_isochrones.to_json('../../data/Isochrone_data/car_isochrones.json', orient='records', lines=True, force_ascii=False)

Fetched isochrone for interval 10M, MRT: ESPLANADE MRT STATION
Fetched isochrone for interval 10M, MRT: PAYA LEBAR MRT STATION


IndexError: list index out of range

In [10]:
bicycle_isochrones = pd.read_json('../../data/Isochrone_data/bicycle_isochrones.json', orient='records', lines=True)
bicycle_isochrones.head()

,MRT.Name,Latitude,Longitude,isochrone_10M,isochrone_15M,isochrone_20M,isochrone_25M,isochrone_30M,isochrone_35M,isochrone_40M,isochrone_45M,isochrone_50M,isochrone_55M,isochrone_60M
0,ESPLANADE MRT STATION,1.293658,103.855081,"[[103.861, 1.284867], [103.861722, 1.286027], ...","[[103.841367, 1.289686], [103.841584, 1.289203...","[[103.831873, 1.288905], [103.832188, 1.288877...","[[103.888542, 1.29323], [103.888929, 1.293201]...","[[103.869582, 1.3302019999999999], [103.868636...","[[103.812166, 1.2781959999999999], [103.812398...","[[103.809332, 1.262772], [103.810599, 1.262453...","[[103.813108, 1.337934], [103.813677, 1.338025...","[[103.806029, 1.338051], [103.80538, 1.338502]...","[[103.806029, 1.338051], [103.80538, 1.338502]...","[[103.806029, 1.338051], [103.80538, 1.338502]..."
1,PAYA LEBAR MRT STATION,1.317199,103.892365,"[[103.889556, 1.307294], [103.889875, 1.307317...","[[103.877473, 1.325003], [103.876758, 1.324139...","[[103.901946, 1.2953999999999999], [103.903282...","[[103.858745, 1.2968709999999999], [103.860358...","[[103.863491, 1.287579], [103.864052, 1.287626...","[[103.872771, 1.279147], [103.873047, 1.279572...","[[103.89785, 1.378247], [103.897584, 1.379152]...","[[103.872771, 1.279147], [103.873047, 1.279572...","[[103.829497, 1.336703], [103.829566, 1.336401...","[[103.829843, 1.273725], [103.829869, 1.2737],...","[[103.829843, 1.273725], [103.829869, 1.2737],..."
2,DHOBY GHAUT MRT STATION,1.298655,103.846194,"[[103.833806, 1.29983], [103.833988, 1.299424]...","[[103.850876, 1.283103], [103.85223, 1.283638]...","[[103.819005, 1.292228], [103.819595, 1.291808...","[[103.872969, 1.296341], [103.874022, 1.295847...","[[103.810599, 1.283574], [103.811982, 1.283408...","[[103.810599, 1.26787], [103.810721, 1.267758]...","[[103.807995, 1.263234], [103.8088, 1.262982],...","[[103.892113, 1.341375], [103.891971, 1.342099...","[[103.891038, 1.348936], [103.89062, 1.348876]...","[[103.891038, 1.348936], [103.89062, 1.348876]...","[[103.891038, 1.348936], [103.89062, 1.348876]..."
3,DAKOTA MRT STATION,1.308548,103.889065,"[[103.876785, 1.303843], [103.876851, 1.303841...","[[103.86878, 1.304214], [103.869766, 1.303662]...","[[103.885931, 1.334911], [103.88592, 1.334783]...","[[103.857381, 1.291005], [103.858034, 1.290733...","[[103.891038, 1.348776], [103.890094, 1.34835]...","[[103.898297, 1.357111], [103.897485, 1.357569...","[[103.898754, 1.3683589999999999], [103.898747...","[[103.829973, 1.272056], [103.831668, 1.273447...","[[103.824297, 1.278972], [103.824471, 1.27813]...","[[103.824297, 1.278972], [103.824471, 1.27813]...","[[103.921507, 1.374358], [103.921501, 1.374475..."
4,LAVENDER MRT STATION,1.307378,103.862768,"[[103.866625, 1.300138], [103.867741, 1.300542...","[[103.863903, 1.28979], [103.864052, 1.289684]...","[[103.844063, 1.295915], [103.844374, 1.295568...","[[103.840038, 1.288104], [103.84054, 1.287061]...","[[103.830175, 1.290805], [103.830389, 1.29043]...","[[103.819432, 1.290856], [103.819595, 1.290773...","[[103.834251, 1.351184], [103.83387, 1.350467]...","[[103.902546, 1.342812], [103.902214, 1.343898...","[[103.806451, 1.273721], [103.80648, 1.273687]...","[[103.919824, 1.376512], [103.920608, 1.378072...","[[103.902546, 1.342812], [103.902214, 1.343898..."
